# 🧠 Köpek Deri Hastalıkları Sınıflandırması (Gerçek Eğitim)
**MobilNetV2 tabanlı model eğitimi ve eğitim verilerinin dışa aktarımı için kullanılır.**
Bu not defteri sonunda gerçek doğruluk ve kayıp verilerini elde edeceğiz.

In [8]:
# 📦 Gerekli Kütüphaneleri Kur
!pip install tensorflow matplotlib pandas

## 📁 Google Drive'ı Bağlayın ve ZIP Dosyasını Yükleyin

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# ✅ ZIP dosyasını açın (örnek yol: /content/drive/MyDrive/dogs.zip)
import zipfile
zip_path = '/content/drive/MyDrive/dogs.zip'  # <- Dosya adını kendine göre değiştir
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dog_dataset')

## 🧠 Modeli Eğit ve Doğruluk/Kayıp Verilerini Kaydet

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import pandas as pd
import os

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10

train_dir = '/content/dog_dataset/train'
val_dir = '/content/dog_dataset/valid'

train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical')

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical')

mobilenet = MobileNetV2(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet')
mobilenet.trainable = False

model = models.Sequential([
    mobilenet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Eğitimi başlat
history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS)

# Sonuçları dataframe'e çevir ve dışa aktar
history_df = pd.DataFrame(history.history)
history_df.to_csv('/content/history.csv', index=False)
print('Eğitim tamamlandı! Gerçek veriler /content/history.csv olarak kaydedildi.')

Found 3022 images belonging to 6 classes.
Found 860 images belonging to 6 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.5493 - loss: 1.3509 - val_accuracy: 0.7721 - val_loss: 0.6400
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.7695 - loss: 0.6625 - val_accuracy: 0.8163 - val_loss: 0.5211
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 176s 2s/step - accuracy: 0.8147 - loss: 0.5166 - val_accuracy: 0.8372 - val_loss: 0.4797
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.8417 - loss: 0.4468 - val_accuracy: 0.8547 - val_loss: 0.4609
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.8868 - loss: 0.3396 - val_accuracy: 0.8640 - val_loss: 0.4174
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.9071 - loss: 0.2952 - val_accuracy: 0.8686 - val_loss: 0.3965
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.9178 - loss: 0.2448 - val_accuracy: 0.8616 - val_loss: 0.4109
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.9278 - loss: 0.2205 - val_accuracy: 0.8860 - v

In [14]:
# Eğitim tamamlandıktan sonra hem history.csv hem de dog_skin_model.h5 kaydedilecek!

# 1. Eğitim geçmişini CSV olarak kaydet
import pandas as pd

history_df = pd.DataFrame(history.history)
history_df.to_csv('/content/history.csv', index=False)
print("✅ Eğitim geçmişi history.csv dosyasına kaydedildi.")

# 2. Eğitilmiş modeli .h5 dosyası olarak kaydet
model.save('/content/dog_skin_model.h5')
print("✅ Model dog_skin_model.h5 dosyasına kaydedildi.")

✅ Eğitim geçmişi history.csv dosyasına kaydedildi.
✅ Model dog_skin_model.h5 dosyasına kaydedildi.
